In [21]:
#Loading libraries and setting spark context
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark import SparkConf, SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithLBFGS 
from pyspark.mllib.classification import LogisticRegressionWithSGD
from sklearn import metrics
from sklearn.metrics import roc_auc_score

from sklearn import metrics
from numpy import array
import numpy as np
#sc = SparkContext(appName="Classification")

In [34]:
#Importing the winedata to a RDD
dataFile = sc.textFile("C:\Users\Bellamkonda\Desktop\Spark\wine1.csv")

In [35]:
#Viewing the first 10 records in the datafile RDD
dataFile.take(10)

[u'0,7 0.27 0.36 20.7 0.045 45 170 1.001 3 0.45 8.8',
 u'0,6.3 0.3 0.34 1.6 0.049 14 132 0.994 3.3 0.49 9.5',
 u'0,8.1 0.28 0.4 6.9 0.05 30 97 0.9951 3.26 0.44 10.1',
 u'0,7.2 0.23 0.32 8.5 0.058 47 186 0.9956 3.19 0.4 9.9',
 u'0,7.2 0.23 0.32 8.5 0.058 47 186 0.9956 3.19 0.4 9.9',
 u'0,8.1 0.28 0.4 6.9 0.05 30 97 0.9951 3.26 0.44 10.1',
 u'0,6.2 0.32 0.16 7 0.045 30 136 0.9949 3.18 0.47 9.6',
 u'0,7 0.27 0.36 20.7 0.045 45 170 1.001 3 0.45 8.8',
 u'0,6.3 0.3 0.34 1.6 0.049 14 132 0.994 3.3 0.49 9.5',
 u'0,8.1 0.22 0.43 1.5 0.044 28 129 0.9938 3.22 0.45 11']

In [36]:
#Parsing function and creating a labeled point for dependent and independent variables
def line_to_array(line):
    string_array = line.split(',')
    return LabeledPoint(string_array[0], string_array[1].split(' '))

In [37]:
#Using map function to parse the data
parsedData = dataFile.map(line_to_array)

In [38]:
#Observing the first 10 records in the parsedData RDD
parsedData.take(10)

[LabeledPoint(0.0, [7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8]),
 LabeledPoint(0.0, [6.3,0.3,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5]),
 LabeledPoint(0.0, [8.1,0.28,0.4,6.9,0.05,30.0,97.0,0.9951,3.26,0.44,10.1]),
 LabeledPoint(0.0, [7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.4,9.9]),
 LabeledPoint(0.0, [7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.4,9.9]),
 LabeledPoint(0.0, [8.1,0.28,0.4,6.9,0.05,30.0,97.0,0.9951,3.26,0.44,10.1]),
 LabeledPoint(0.0, [6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6]),
 LabeledPoint(0.0, [7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8]),
 LabeledPoint(0.0, [6.3,0.3,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5]),
 LabeledPoint(0.0, [8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,11.0])]

In [39]:
#using randomSplit function to split data into trainingRDD and testRDD 
(trainingData, testData) = parsedData.randomSplit([0.7, 0.3])

In [40]:
#printing count of trainingRDD
print trainingData.count()

3457


In [42]:
#printing count of testRDD
print testData.count()

1441


In [55]:
# Build the model SVM with SGD_l1 classifier
model_sgd = SVMWithSGD.train(trainingData,step=0.01,regParam=0.01,iterations=1000)

In [61]:
# Evaluating the model on test data
labelsAndPreds = testData.map(lambda p: (p.label, model_sgd.predict(p.features)))
testErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedData.count())
print("Test Error = " + str(testErr))



Test Error = 0.064311964067


In [67]:
# Build the model LogisticRegressionWithLBFGS

model_lbfgs = LogisticRegressionWithLBFGS.train(trainingData,iterations=1200,regParam=0.1,tolerance=0.0001)

In [68]:
# Evaluating the model on test data
labelsAndPreds = testData.map(lambda p: (p.label, model_lbfgs.predict(p.features)))
testErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedData.count())
print("Test Error = " + str(testErr))


Test Error = 0.0628828093099


In [76]:
# Build the model LogisticRegressionWithSGD_l2

model_sgd_l2 = LogisticRegressionWithSGD.train(trainingData,iterations=1000,regParam=0.1,step=0.001)

In [77]:
# Evaluating the model on test data
labelsAndPreds = testData.map(lambda p: (p.label, model_sgd_l2.predict(p.features)))
testErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedData.count())
print("Test Error = " + str(testErr))


Test Error = 0.064311964067


In [79]:
# Build the model LogisticRegressionWithSGD_l1

model_sgd_l1 = LogisticRegressionWithSGD.train(trainingData,iterations=500,step=0.1,regParam=0.01,regType='l1')

In [56]:
# Evaluating the model on test data
labelsAndPreds = testData.map(lambda p: (p.label, model_sgd_l1.predict(p.features)))
testErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedData.count())
print("Test Error = " + str(testErr))

Test Error = 0.065741118824


In [59]:
# Build the model LogisticRegressionWithSGD_none

model_sgd_none = LogisticRegressionWithSGD.train(trainingData,iterations=500,step=0.1,regParam=0.01,regType='none')

In [60]:
# Evaluating the model on test data
labelsAndPreds = testData.map(lambda p: (p.label, model_sgd_none.predict(p.features)))
testErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedData.count())
print("Test Error = " + str(testErr))

Test Error = 0.0667619436505
